In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/l2sp' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes)
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.010000
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:5.302093982696533 | top1:37.5
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:2.492322117090225 | top1:48.4375


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


107/107 | Total:0:00:17 | ETA:0:00:00 | Loss:1.0991836123377363 | top1:33.2149543762207
26/26 | Total:0:00:14 | ETA:0:00:00 | Loss:0.4240302134018678 | top1:77.4102554321289

Epoch: [2 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.8563136458396912 | top1:65.625
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.8413570324579874 | top1:53.645835876464844

Epoch: [3 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.686688244342804 | top1:56.25
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.7197591463724772 | top1:54.16666793823242

Epoch: [4 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.643040120601654 | top1:65.625
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.720857560634613 | top1:51.5625

Epoch: [5 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.6623460054397583 | top1:65.625
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.7083104054133097 | top1:54.6875

Epoch: [6 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.6789669990

1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.4652290940284729 | top1:78.125
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.4406687368949254 | top1:80.20833587646484

Epoch: [46 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.4867010712623596 | top1:75.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.4733746250470479 | top1:78.125

Epoch: [47 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.6163246035575867 | top1:56.25
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.4411159058411916 | top1:76.04167175292969

Epoch: [48 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.4143350422382355 | top1:78.125
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.45321492354075116 | top1:73.4375

Epoch: [49 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.37018707394599915 | top1:81.25
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.3985982835292816 | top1:81.25

Epoch: [50 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.5608271360397339 | top1:75.0
7/7

1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.2651188373565674 | top1:93.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.3906063536802928 | top1:84.89583587646484

Epoch: [89 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.22972482442855835 | top1:90.625
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.25453342000643414 | top1:88.02083587646484

Epoch: [90 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.1862226128578186 | top1:93.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.22367776185274124 | top1:92.70833587646484

Epoch: [91 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.3710562288761139 | top1:75.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.24249289433161417 | top1:89.0625

Epoch: [92 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.1909230500459671 | top1:93.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.21117315938075384 | top1:90.10417175292969

Epoch: [93 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0742089

...

107/107 | Total:0:00:32 | ETA:0:00:00 | Loss:0.5342126849377267 | top1:87.85670471191406
26/26 | Total:0:00:15 | ETA:0:00:00 | Loss:2.412921978877141 | top1:54.32051086425781

Epoch: [4802 | 5000] LR: 0.000799
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0010232329368591309 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0012819233040014903 | top1:100.0

Epoch: [4803 | 5000] LR: 0.000008
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.00019346922636032104 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.00544951343908906 | top1:100.0

Epoch: [4804 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0009232610464096069 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.012906512866417566 | top1:99.47917175292969

Epoch: [4805 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.005352068692445755 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.003054009440044562 | top1:100.0

Epoch: [4806 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 

7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.02468740474432707 | top1:99.47917175292969

Epoch: [4845 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0006384998559951782 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.007196352196236451 | top1:100.0

Epoch: [4846 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.000631183385848999 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002858694332341353 | top1:100.0

Epoch: [4847 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.0004072263836860657 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.011307111475616693 | top1:98.95833587646484

Epoch: [4848 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.00047332048416137695 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0015776101499795914 | top1:100.0

Epoch: [4849 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.00018601864576339722 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.00

1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.00033480674028396606 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0010751187801361084 | top1:100.0

Epoch: [4888 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0004941299557685852 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0083307313422362 | top1:99.47917175292969

Epoch: [4889 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.00011970847845077515 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.023207254086931545 | top1:99.47917175292969

Epoch: [4890 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.031019702553749084 | top1:96.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.00564738300939401 | top1:99.47917175292969

Epoch: [4891 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0008741989731788635 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0057385948797067 | top1:99.47917175292969

Epoch: [4892 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ET

1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.008425522595643997 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0023156609386205673 | top1:100.0

Epoch: [4930 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.00026389211416244507 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0028289281763136387 | top1:100.0

Epoch: [4931 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0001838207244873047 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.003008030354976654 | top1:100.0

Epoch: [4932 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.0002700239419937134 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.010784757323563099 | top1:98.95833587646484

Epoch: [4933 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.010818742215633392 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0032875823477904 | top1:100.0

Epoch: [4934 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0001769810914993

7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0006872980544964472 | top1:100.0

Epoch: [4972 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0005691275000572205 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.016831930726766586 | top1:98.95833587646484

Epoch: [4973 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.00032316893339157104 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002269894350320101 | top1:100.0

Epoch: [4974 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.00016678869724273682 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0023009683936834335 | top1:100.0

Epoch: [4975 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0007818564772605896 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0007816000531117121 | top1:100.0

Epoch: [4976 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.003982551395893097 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.00202037952